In [1]:
from utils.flurico import FluRiCoAnalysis

import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import tqdm
from joblib import load,dump

np.random.seed(888)

In [2]:
# Load your data
df_mb = pd.read_csv('data_example/clr.csv',index_col=0) # microbiome relative abundance should be processed by Centered log-ratio transformation
df_mt = pd.read_csv('data_example/metabolites.csv',index_col=0)

In [3]:
# Load co-abundance data
df_mb_corr = {}
for i in ['HC','MIA','IA']:
    df_mb_corr[i] = pd.read_csv(f'data_example/sparcc/{i}_median_correlation.tsv',sep='\t',index_col=0)


In [4]:
# Define your analyzer
analyzer = FluRiCoAnalysis(microbe_data=df_mb, metabolite_data=df_mt, 
                           group_labels=['HC','MIA','IA'], microbe_coab_data = df_mb_corr)


In [5]:
all_scores = analyzer.calculate_all_scores()

Calculating Fluctuation...
Calculating microbe fluctuation...


100%|██████████| 5113/5113 [00:15<00:00, 322.70it/s]


Total 0 features do not have sufficient variability data for Levene test
Calculating metabolite fluctuation...


100%|██████████| 763/763 [00:02<00:00, 341.10it/s]


Total 0 features do not have sufficient variability data for Levene test
Applying HC group microbe coabundance...
Calculating HC group metabolite average correlation...
Applying MIA group microbe coabundance...
Calculating MIA group metabolite average correlation...
Applying IA group microbe coabundance...
Calculating IA group metabolite average correlation...
Calculating remote interaction...
Calculating HC group microbe-metabolite remote interaction...


...

MIA - metabolite: 100%|██████████| 763/763 [01:32<00:00,  8.29it/s]


Calculating IA group microbe-metabolite remote interaction...


IA - microbe: 100%|██████████| 5113/5113 [03:17<00:00, 25.93it/s]


Calculating IA group metabolite-microbe remote interaction...


IA - metabolite: 100%|██████████| 763/763 [01:31<00:00,  8.36it/s]


In [6]:
# filter top 20% feature for microbiome and metabolome in each cohort
common_features = analyzer.query_common_features(top_percent=0.2)

In [7]:
def get_deepest_keys(d, prefix=None):

    if prefix is None:
        prefix = []
    if isinstance(d, dict) and d: 
        for k, v in d.items():
            yield from get_deepest_keys(v, prefix + [k])
    else:
        yield prefix

In [8]:
# common_features dict
for path in get_deepest_keys(common_features):
    print(" -> ".join(path))

# common_to_all means the spotlight feature
# unique_to_each means the core feature
# pairwise_common means with 2 set in common

microbe -> common_to_all
microbe -> unique_to_each -> HC
microbe -> unique_to_each -> MIA
microbe -> unique_to_each -> IA
microbe -> pairwise_common -> HC_and_MIA
microbe -> pairwise_common -> HC_and_IA
microbe -> pairwise_common -> MIA_and_IA
metabolite -> common_to_all
metabolite -> unique_to_each -> HC
metabolite -> unique_to_each -> MIA
metabolite -> unique_to_each -> IA
metabolite -> pairwise_common -> HC_and_MIA
metabolite -> pairwise_common -> HC_and_IA
metabolite -> pairwise_common -> MIA_and_IA


In [9]:
# save the results
analyzer.save_results(output_dir="results")
dump(analyzer,'results/analyzer.class')
dump(common_features,'results/common_features.class')

Integrated scores saved to results/microbe_summary_HC.csv
Integrated scores saved to results/microbe_summary_MIA.csv
Integrated scores saved to results/microbe_summary_IA.csv
Fluctuation analysis p-values saved to results/microbe_fluctuation_p_values.csv
Integrated scores saved to results/metabolite_summary_HC.csv
Integrated scores saved to results/metabolite_summary_MIA.csv
Integrated scores saved to results/metabolite_summary_IA.csv
Fluctuation analysis p-values saved to results/metabolite_fluctuation_p_values.csv
All results saved to results directory


['results/common_features.class']

In [10]:
print(f"Spotlight microbes:{len(common_features['microbe']['common_to_all'])}, Spotlight metabolites:{len(common_features['metabolite']['common_to_all'])}")
print(
    f"HC core microbes:{len(common_features['microbe']['unique_to_each']['HC'])}, "
    f"MIA core microbes:{len(common_features['microbe']['unique_to_each']['MIA'])}, "
    f"IA core microbes:{len(common_features['microbe']['unique_to_each']['IA'])}")
print(
    f"HC core metabolites:{len(common_features['metabolite']['unique_to_each']['HC'])}, "
    f"MIA core metabolites:{len(common_features['metabolite']['unique_to_each']['MIA'])}, "
    f"IA core metabolites:{len(common_features['metabolite']['unique_to_each']['IA'])}")
print(
    f"HC overlap microbes:{len(common_features['microbe']['pairwise_common']['HC_and_MIA'])}, "
    f"MIA overlap microbe:{len(common_features['microbe']['pairwise_common']['HC_and_IA'])}, "
    f"IA overlap microbe:{len(common_features['microbe']['pairwise_common']['MIA_and_IA'])}, "
    f"in total:{len(common_features['microbe']['pairwise_common']['HC_and_MIA'])+len(common_features['microbe']['pairwise_common']['HC_and_IA'])+len(common_features['microbe']['pairwise_common']['MIA_and_IA'])}")
print(
    f"HC overlap metabolites:{len(common_features['metabolite']['pairwise_common']['HC_and_MIA'])}, "
    f"MIA overlap metabolites:{len(common_features['metabolite']['pairwise_common']['HC_and_IA'])}, "
    f"IA overlap metabolites:{len(common_features['metabolite']['pairwise_common']['MIA_and_IA'])}, "
    f"in total:{len(common_features['metabolite']['pairwise_common']['HC_and_MIA'])+len(common_features['metabolite']['pairwise_common']['HC_and_IA'])+len(common_features['metabolite']['pairwise_common']['MIA_and_IA'])}")

Spotlight microbes:388, Spotlight metabolites:38
HC core microbes:299, MIA core microbes:311, IA core microbes:308
HC core metabolites:76, MIA core metabolites:66, IA core metabolites:64
HC overlap microbes:166, MIA overlap microbe:169, IA overlap microbe:157, in total:492
HC overlap metabolites:18, MIA overlap metabolites:20, IA overlap metabolites:30, in total:68
